In [1]:
%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import pandas as pd

import distance
import nltk
#nltk.download('stopwords')

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

import copy
from openpyxl import load_workbook

In [2]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-14-13-28-20_1stPart_2' : [2], # cw_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # cw_session3_2_part2
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    #'2019-02-18-10-28-35_2' : [0],               # ls2_session4 # picture not described
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-30-11-22-25_1' : [4, 6, 7] # ys, session 4
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11]   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
   
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [3]:
dict_keySelection_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 3, 5], # ys_session2 
}

dict_keySelection_WritingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2   
}

# normally, reading part of trial ends when people look at the keyboardwithphrases. For some trials, this is not done,
# as the reading is done, and the trial is accidentally skipped, and written in the next trial. Here, the trial number 
# given will have the reading time ending as sleep, and not keyboard with phrases. 
dict_keyboardNotChange_ReadingTrials = {
    "2019-01-16-15-42-51_2ndPart_1" : 0, # ys_session2 
}

dict_keySelection_firstSleepNotCounted = {
    "2019-01-28-14-50-41_2" : (0, 2), # bh1_session1 -- 3rd sleep activation to be counted
    "2019-02-19-10-56-43_2ndPart_1" : 2  # mn_session3 -- 3rd sleep activation is to be counted
}

In [4]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [5]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [6]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [7]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [8]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [9]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_dwellTimeOrig_not800:
        TimeDwellOrig = dict_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                # this is a quick-fix for something weird happening -- two consecutive 'DecreaseDwellTime' selected,
                # but the second one actually does not take effect on the keyboard, and needs to be not counted
                if session_folder_name == '2019-01-17-15-03-40_1' and nKey == 147:
                    continue
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [10]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    

    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
            
    return trials    

In [11]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [12]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [13]:
def stimPhrasesEdit(PhraseLog, full_path):
   
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
        
    phraseStim_PhrasesReduced, phraseStim_timeReduced = zip(*[(x[0], PhraseLog[phraseStim_Phrases.index(x[0])][0]) for x in groupby(phraseStim_Phrases)])
    
    PhraseLogReduced = [[phraseStim_timeReduced[i], phraseStim_PhrasesReduced[i]] for i in range(0, len(phraseStim_PhrasesReduced))]
    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if PhraseLogReduced[0][1] == 'phraseText':
        del PhraseLogReduced[0]

    # Here, we want only the sentences typed
    notSentencesToType = list()
    for index in range(0,len(PhraseLogReduced)):
        sentence = PhraseLogReduced[index][1]
        if 'Svar på følgende spørgsmål' in sentence or 'Answer the question:' in sentence or 'What is the complete name of your university?' in sentence or '(give a score between 1 and 7)' in sentence or sentence == '':
            notSentencesToType.append(index)
         
    
    for index in sorted(notSentencesToType, reverse=True):
        del PhraseLogReduced[index]
    
    if session_folder_name in dict_phraseStim:
        #print('session in stim phrases found')
        index_to_be_removed = dict_phraseStim[session_folder_name]
    else:
        index_to_be_removed = []
        
    #print(index_to_be_removed)
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del PhraseLogReduced[index]
        
    return PhraseLogReduced

In [14]:
def scratchPadPhraseEdit(phraseUser, subjName, full_path, picture):
    phraseUserEnd = list()
    
    
    
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if phraseUser[row_ind-1][1] != 'scratchPadText':
                if len(phraseUser[row_ind-1][1])>2: # this also removes any answers on the difficulty of the sentence
                    phraseUserEnd.append(phraseUser[row_ind-1])
                    #print(phraseUser[row_ind-1])
                    
    #print(' (1) phrases reduced to :       ', phraseUserEnd)
    # remove first two trials of baseline question and text composition
    if 'not_described' in picture or '2019-01-14-14-58-30' in full_path or '2019-02-06-12-37-45_2' in full_path or '2019-02-18-10-28-35_2' in full_path: 
        # yss has not copied one of the sentences, aq_session1_2 has not written the baseline question, ls2_session4_2 did not describe the picture
        del phraseUserEnd[0]
    else:
        del phraseUserEnd[0:2]
      
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dict_phraseUser:
        index_to_be_removed = dict_phraseUser[session_folder_name]
        #print('session in user phrases found')
    else:
        index_to_be_removed = []
        
    
    if index_to_be_removed:
        for index in sorted(index_to_be_removed, reverse=True):
            del phraseUserEnd[index]
        
    
    #print(' (2) phrases reduced to:       ', phraseUserEnd)
    
    # remove the initial rating of complexity, if they have written it:
    for index in range(0,len(phraseUserEnd)):
        sentence = phraseUserEnd[index][1]
        if sentence[0].isdigit():
            
            # if there is also a space after the digit:
            
            if sentence[1] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][1:]
            
        elif sentence[1].isdigit():
            if sentence[2] == ' ':
                phraseUserEnd[index][1] = phraseUserEnd[index][1][3:]
            else:
                phraseUserEnd[index][1] = phraseUserEnd[index][1][2:]
    
    #print(' (3) phrases reduced to:       ', phraseUserEnd)
    
    return phraseUserEnd

In [15]:
def ComputeCorrectLetters(PhraseStim, PhraseUserEnd):
    
    if len(PhraseUserEnd)==len(PhraseStim):
        list_nCorrectLetters = list()
        
        for element in range(0,len(PhraseStim)):
            sentence_phraseStim = PhraseStim[element][1]
            sentence_phraseUser = PhraseUserEnd[element][1]
            
            #print(sentence_phraseStim)
            #print(sentence_phraseUser)
            
            # edit sentence if it has a period at the end:
            #if sentence_phraseStim[-1] == '.':
            #    sentence_phraseStim = sentence_phraseStim[:-1]
            #if sentence_phraseUser[-1] != '.':
            #    sentence_phraseUser = sentence_phraseUser[:-1]
                
            # if pound sign in the sentence:
            # check first if it is in the phraseStim.
            if '£50' in sentence_phraseStim:
                # now if some form of currency (pound/kr/kr.) is typed, that will also be added to the phraseStim
                if 'pounds' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 pounds')
                    #print('new', sentence_phraseStim)
                elif 'pound' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 pound')
                    #print('new', sentence_phraseStim)
                
                elif '50kr.' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50kr.')
                    #print('new', sentence_phraseStim)
                elif '50kr' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50kr')
                    #print('new', sentence_phraseStim)
                
                elif '50 kr.' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 kr.')
                    #print('new', sentence_phraseStim)
                elif '50 kr' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 kr')
                    #print('new', sentence_phraseStim)
                
                elif 'L50' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', 'L50')
                    #print('new', sentence_phraseStim)
                
                else:
                    sentence_phraseStim = sentence_phraseStim.replace('£', '')
                    #print('new', sentence_phraseStim)
                    
            """
            # if people write [?] instead of the complete sentence
            if '[?]' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('[?]', '')
            
            # if people write .../..../..... instead of the complete sentence
            
            if '.....' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('.....', '')
            elif '....' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('....', '')
            elif '...' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('...', '')
            elif '..' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('..', '')
            """   
            
            print(sentence_phraseStim)
            print(sentence_phraseUser)
            print('\n')
            
            nCorrectLetters = max(len(sentence_phraseStim), len(sentence_phraseUser)) - distance.levenshtein(sentence_phraseStim.lower(), sentence_phraseUser.lower())
            #nCorrectLetters = len(sentence_phraseUser) - distance.levenshtein(sentence_phraseStim.lower(), sentence_phraseUser.lower())
            
            list_nCorrectLetters.append(nCorrectLetters)
            #print('Number of correct letters typed: ', nCorrectLetters, '   and number of letters to be typed: ', len(sentence_phraseStim))
            #print('\n')
            
    
    else:
        print('stimulation phrases and user phrases do not match')
        print('stimulation phrases:    ', PhraseStim)
        print('user phrases:    ', PhraseUserEnd)
        
    return list_nCorrectLetters

In [16]:
def FindExptStartEndTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            startTimeTrial = endTimeTrial + datetime.timedelta(seconds=5)
            
            #print('start: ', startTimeTrial)
            timeTrialDict['start'].append(startTimeTrial)
        
    del timeTrialDict['start'][-1]
    
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    timeTrialDict_copy = copy.deepcopy(timeTrialDict)
    
    # separate the reading and writing trials for some participants who read in the actual trial, but write in the next
    # trial
    if session_folder_name in dict_keySelection_ReadingTrials:
        # check the reading and writing separate dictionaries
        print('reading and writing sessions are separate')
        
        #print(len(timeTrialDict['start']))
        # writing trials - 
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_WritingTrials, timeTrialDict, replacingList)
        #print(len(timeTrialDict_copy['start']))
        
        # reading trials
        timeTrialDict_reading = findAndRemoveTrials(session_folder_name, dict_keySelection_ReadingTrials, timeTrialDict_copy, replacingList)
    else:
        # some participants skip some sentences, and then it affects the scoreQuestions too. Remove the skipped sentences or 
        # remove the score questions 
        # for these participants, reading and writing trials are the same
        
        #print('same reading and writing trials')
        scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
        timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
        
        # most of the skipped sentences are removed, but for those that are not removed
        timeTrialDict_writing = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
        
        timeTrialDict_reading = timeTrialDict_writing 
        
    
          
    return timeTrialDict_reading, timeTrialDict_writing 

In [17]:
def FindReadingPartsOfTrial_inKeysSelected(EventTrials_reading, KeysSelected_new, full_path, userKeys):
    # find the reading end in the pupil size data
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    userKeysTimeStr = [key[0] for key in userKeys]
    userKeysTime = timeConversion(userKeysTimeStr)
    
    EventReading = dict()    
    EventReading['start'] = list()
    EventReading['end'] = list()
    
    EventReading_index = dict()    
    EventReading_index['start'] = list()
    EventReading_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_reading['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        
        EventReading['start'].append(KeysSelected_time[startTrialInd]+datetime.timedelta(seconds=5)) # start time needs 
        #to start 5s later, which is when the phrase is visible
        EventReading_index['start'].append(startTrialInd) 
        
        
        #print(ind, EventReading['start'][-1], EventReading_index['start'][-1])
        
        endTrialTime = EventTrials_reading['end'][ind]
        endTrialInd = KeysSelected_time.index(endTrialTime)
        
        keysSelected_trial = KeysSelected_keys[startTrialInd:endTrialInd]
        
        
        for i, key in enumerate(keysSelected_trial):
            if len(key) == 1:
                endReading_keyInd = startTrialInd + keysSelected_trial.index(key) - 1
                break
                    
        endReading_keyTime = KeysSelected_time[endReading_keyInd]
        
        """
        # compute the end of reading time -- which is taken when the keyboard with phrases key is pressed
        if session_folder_name not in dict_keyboardNotChange_ReadingTrials:
            
            for i, key in enumerate(keysSelected_trial):
                if len(key) == 1:
                    endReading_keyInd = startTrialInd + keysSelected_trial.index(key) - 1
                    break
                    
            #endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
            endReading_keyTime = KeysSelected_time[endReading_keyInd]
            
        else:
            if ind == dict_keyboardNotChange_ReadingTrials[session_folder_name]:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('Sleep') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
            else:
                endReading_keyInd = startTrialInd + keysSelected_trial.index('KeyboardWithPhrases') - 1
                endReading_keyTime = KeysSelected_time[endReading_keyInd]
                
        """       
        
        
        userKeyTime, userKeyInd = nearestTimePoint(userKeysTime, KeysSelected_time[endReading_keyInd+1])
        
        # remove the dwell time from end of selecting the first key (letter/number)
        EventReading['end'].append(KeysSelected_time[endReading_keyInd+1]-datetime.timedelta(milliseconds=\
                                    float(userKeys[userKeyInd][-1])))
        
        EventReading_index['end'].append(KeysSelected_time[endReading_keyInd+1])
        
        #print(ind, EventReading['start'][ind], EventReading['end'][ind])
        
        #print(ind)
        #print('reading: ', EventReading['start'][ind], EventReading['end'][ind])
        
    
    
    return EventReading, EventReading_index

In [18]:
def FindWritingPartsOfTrial_inKeysSelected(EventTrials_writing, KeysSelected_new, EventReading):
    
    KeysSelected_timeStr = [key[0] for key in KeysSelected_new]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected_new]
    
    EventWriting = dict()    
    EventWriting['start'] = list()
    EventWriting['end'] = list()
    
    EventWriting_index = dict()    
    EventWriting_index['start'] = list()
    EventWriting_index['end'] = list()
    
    for ind, startTrialTime_afterCoolDown in enumerate(EventTrials_writing['start']):
        
        startTrialTime, startTrialInd = nearestTimePoint(KeysSelected_time, startTrialTime_afterCoolDown)
        
        endTrialTime = EventTrials_writing['end'][ind]
        endTimeReading = EventReading['end'][ind]
        
        
        # for some participants, reading and writing trials are different. So their writing times will not be the end of 
        # the reading time.
        # Regardless, the writing time should start later than when the reading time ends.
        # We choose the starting time for writing as the one that is later than the start time from writing trials
        # and end time from reading trials
        
        if startTrialTime > endTimeReading:
            EventWriting['start'].append(startTrialTime)
            EventWriting_index['start'].append(startTrialInd)
        else:
            EventWriting['start'].append(endTimeReading)
            endTimeReading_keys, endTimeReading_ind = nearestTimePoint(KeysSelected_time, endTimeReading)
            EventWriting_index['start'].append(KeysSelected_time.index(endTimeReading_keys))
        
        EventWriting['end'].append(endTrialTime)
        EventWriting_index['end'].append(KeysSelected_time.index(endTrialTime))
        
        #print(ind)
        #print('writing: ', EventWriting['start'][ind], EventWriting['end'][ind])
        
    
    
    return EventWriting, EventWriting_index     

In [19]:
def EffectiveTimeFromUserKeys(UserKeys, Event, pathOfSession):
    
    UserKeysTimeStr = [val[0] for val in UserKeys]
    UserKeysTime = timeConversion(UserKeysTimeStr)
    
    timeActivation = 250
    
    # session name
    session_folder_name = pathOfSession.split('\\')[-1]
    
    timeTypingList = list()
    
    
    for indTrial, eventStart in enumerate(Event['start']):
        eventEnd = Event['end'][indTrial]
        eventStart_UserKeys, eventStartUserKeysInd = nearestTimePoint(UserKeysTime, eventStart)
        eventEnd_UserKeys, eventEnd_UserKeysInd = nearestTimePoint(UserKeysTime, eventEnd)
        time_trial = 0
        
        keySelectedPrevious = ''
        
        for indUserKey in range(eventStartUserKeysInd, eventEnd_UserKeysInd):
            
            if len(UserKeys[indUserKey][1]) == 1 or UserKeys[indUserKey][1] in list_keysToBeCounted or 'Suggestion' in UserKeys[indUserKey][1]:
                
                if len(UserKeys[indUserKey+1]) < 4 or float(UserKeys[indUserKey][3]) > float(UserKeys[indUserKey+1][3]) or UserKeys[indUserKey][2]==1:
                    
                    #print(UserKeys[indUserKey])
                    
                        
                    # Sometimes, in between selection of a letter, there is a small pause and activation of another letter,
                    # indicated by a '0' activation of the second letter during the first letter selection.
                    # This requires adding only the activation time for the second letter, not the dwell-time for the 
                    # current letter, which will be completed later on
                    
                    if len(UserKeys[indUserKey+1])>3 and len(UserKeys[indUserKey+2])>3:
                        if float(UserKeys[indUserKey][3]) > float(UserKeys[indUserKey+1][3]):
                            if float(UserKeys[indUserKey+1][3]) == 0:
                                if UserKeys[indUserKey+2][1] == UserKeys[indUserKey][1]:
                                    time_trial = time_trial + timeActivation
                                    #print('Only timeActivation added')
                                    if UserKeys[indUserKey][2]==1:
                                        keySelectedPrevious = UserKeys[indUserKey][1]
                                    continue
                    
                    if keySelectedPrevious == UserKeys[indUserKey][1]: # if the key selection is accidentally continued,
                        # activation time should not be counted
                        time_trial = time_trial + float(UserKeys[indUserKey][3])
                        #print('Current dwell-time added', keySelectedPrevious)
                        
                    else:
                        #print('timeActivation added')
                        time_trial = time_trial + float(UserKeys[indUserKey][3]) + timeActivation
                    
                    if UserKeys[indUserKey][2]==1:
                            keySelectedPrevious = UserKeys[indUserKey][1]
                            
        timeTypingList.append(time_trial/1000) # add the time in seconds
        
    
    return timeTypingList

In [20]:
class DataForEveryTrial:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
    

In [21]:
metricComputed = 'CorrectLettersPerEffectiveTime'
print(metricComputed)
dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\IndividualTrials\Performance\Subject_Block_Session_Trial_' + metricComputed +  '.xlsx'

            

j = 0
TimeDwellAvg = 0
#nSub = 0
TypingSpeed = list()
Names = list()

list_typingSpeed_trial = list()
list_typingSpeed_subject = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'af\\1\\' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
                        
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
            
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            
            # find the language of the data:
            if 'Hvad er det fulde navn på dit universitet?' in phraseLog[1][1]: 
                language = 'Danish'
            else:
                language = 'English'
                
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
                
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new, root)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            phraseStim_reduced = stimPhrasesEdit(phraseLog_new, root)
            
            # if it is the 2nd part of the session, picture is not described:
            if '2ndPart' in root:
                picture = 'not_described'
            else:
                picture = 'described'
            
            phraseUserEnd_reduced = scratchPadPhraseEdit(scratchPad_new, subjName, root, picture)
            
            
            # divide complete data into epochs of phrases
            timeStartEndKeys_reading, timeStartEndKeys_writing = FindExptStartEndTimes(keysSelected_new, timeTyping, root)
            
            # use the times of trial start and end, to divide the time of trial into reading and writing
            eventTrialsInKeysSelected_reading, eventTrialsInKeysSelected_readingIndex = \
            FindReadingPartsOfTrial_inKeysSelected(timeStartEndKeys_reading, keysSelected_new, root, \
                                                   userKeys_wDwellTime)
            
            eventTrialsInKeysSelected_writing, eventTrialsInKeysSelected_writingIndex = \
            FindWritingPartsOfTrial_inKeysSelected(timeStartEndKeys_writing, keysSelected_new, eventTrialsInKeysSelected_reading)
            
            # Effective time of typing:
            # Find time of typing only every letter, numbers, space and changing of keyboards
            effective_time = EffectiveTimeFromUserKeys(userKeys_wDwellTime, eventTrialsInKeysSelected_writing, root)
              
            
            #print('stim phrases', phraseStim_reduced)
            #print('user phrases', phraseUserEnd_reduced)
            
            if '1stPart' in root:
                print('1stPart')
                phraseUserEnd_reduced1 = phraseUserEnd_reduced
                phraseStim_reduced1 = phraseStim_reduced
                effective_time1 = effective_time 
                continue
            
            
            
            if '2ndPart' in root:
                print('2ndPart')
                phraseUserEnd_reduced2 = phraseUserEnd_reduced
                phraseStim_reduced2 = phraseStim_reduced
                effective_time2 = effective_time
                
                phraseUserEnd_reduced = phraseUserEnd_reduced1 + phraseUserEnd_reduced2
                phraseStim_reduced = phraseStim_reduced1 + phraseStim_reduced2
                effective_time = effective_time1 + effective_time2
                
                phraseUserEnd_reduced1 = list()
                phraseStim_reduced1 = list()
                effective_time1 = list()
                
            #print(len(phraseUserEnd_reduced), len(phraseStim_reduced))
                
            # check if the number of sentences match in the input and the output
            if len(phraseUserEnd_reduced)!=len(phraseStim_reduced):
                print('needs fixing of number of phrases')
                #print('before', phraseStim_reduced)
                #phraseStim_toMatch, phraseUserEnd_toMatch = MatchPhraseNumbers(phraseStim_reduced, phraseUserEnd_reduced)
                #print('after', phraseStim_toMatch)
            else:
                phraseStim_toMatch = phraseStim_reduced
                phraseUserEnd_toMatch = phraseUserEnd_reduced
                
            
            #for sentence in phraseUserEnd_toMatch:
            #    print(sentence[1])
            
            # Find the number of correct letters typed
            # ComputeCorrectLetters(phraseStim_reduced, phraseUserEnd_reduced)
            nCorrectLetters = ComputeCorrectLetters(phraseStim_toMatch, phraseUserEnd_toMatch)
            #print(nCorrectLetters)
            
            
            if len(nCorrectLetters) == len(effective_time):
                for i in range(0, len(nCorrectLetters)):
                    
                    list_typingSpeed_trial.append((nCorrectLetters[i]*60)/(effective_time[i]*5))
            else:
                print('nCorrectLetters and effective_time have unequal lengths')
            
            
            dataToSave = DataForEveryTrial()
            dataToSave.subjectID = subjAndSessionName.split('__')[0]
            dataToSave.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave.sessionNumber = subjAndSessionName[-1]
            dataToSave.variable = metricComputed
            dataToSave.dataForTrial = list_typingSpeed_trial
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            dataToSave.AddToFile()
            
            
            
            
            #list_typingSpeed_subject.append(np.mean(list_typingSpeed_trial))
            list_typingSpeed_trial = list()
            


CorrectLettersPerEffectiveTime
subject and session name:  ac__1__2019-02-11-11-18-30_1
I feel I need more help in some areas before I implement it fully.
I feel I need more help in some areas before I implement it fully.


Sir,  your wit ambles well,  it goes easily.
Sir,  your wit ambles well,  it goes easily.


Set the alarm for 5am.
Set the alarm for 5a.m.


As I said,  I believe that fear of death is the root of all religions.
As I said,  I believe that fear of death is the root for all religions.


Can easily be raised from seed.
Can easily be raised from seed.


  subjectID block session  trial  CorrectLettersPerEffectiveTime
0        ac     1       1      0                       11.817320
1        ac     1       1      1                       11.857498
2        ac     1       1      2                       11.582643
3        ac     1       1      3                       11.777522
4        ac     1       1      4                       14.518793
subject and session name:  ac__1__2

subject and session name:  ac__4__2019-02-18-14-59-21_1
Representations of a Shrine on a Mycenaean Chariot Krater from Kalavasos-Ayios Dhimitrios,  Cyprus.
Representations of a Shrine on a Mycenaean Chariot Krater from Kalavasos-Ayios Dhimitrios,  Cyprus.


An opportunity had arisen to include the site in the proposed sale of the adjacent land where the Territorial Army Voluntary Reserve Army (TAVRA) was located.
An opportunity had arisen to include the site in the proposed sale of the adjacent land where the Territory Army Voluntary Reserve  Army (TAVRA) was located.


Beryllium sulphate(VI) is the only sulphate(VI) to decompose at comparable temperatures.
Beryllium Sulphate(VI) is the only sulphate to decompose at comparable temperatures.


Resources helps you find exciting,  useful and interesting books and other printed materials,  CD ROM's,  videos and television and radio programmes that relate to the UK Primary Science Curriculum.
Resources helps you find exciting,  useful and i

How can you over take the person coming last?
How can you overtake the person coming last?


  subjectID block session  trial  CorrectLettersPerEffectiveTime
0        af     1       2      0                       16.659442
1        af     1       2      1                       26.544750
2        af     1       2      2                       35.576471
3        af     1       2      3                       28.285714
4        af     1       2      4                       32.816327
5        af     1       2      5                       34.863158
6        af     1       2      6                       34.622951
subject and session name:  af__2__2019-01-17-15-03-40_1
Many return year after year,  to get their publications bound.
Many return year after year,  to get their finances bound.


You'll have to ask them.
You'll have to ask them


She's completely real that way.
She's completely real that way.


Christians also assert that the Gospel relates not only to what Jesus said but also to wha

subject and session name:  af__5__2019-02-27-15-08-32_1
Resources helps you find exciting,  useful and interesting books and other printed materials,  CD ROM's,  videos and television and radio programmes that relate to the UK Primary Science Curriculum.
Resources help you finding exciting and useful printed materials,  CD-ROMs,  videos,  related to UK Science Curriculum


An opportunity had arisen to include the site in the proposed sale of the adjacent land where the Territorial Army Voluntary Reserve Army (TAVRA) was located.
An opportunity had arisen to include the site in the proposed sale of the adjacent site where the Territorial Army Volunteer Reserve Army (TAVRA) is located.


Representations of a Shrine on a Mycenaean Chariot Krater from Kalavasos-Ayios Dhimitrios,  Cyprus.
Representations of a Shrine on a Mycenaean Chariot Krater in Kalavasos-Ayios Dhimitrios,  Cyprus.


She has a BA Hons and MA in Modern History from Royal Holloway,  University of London and an MA in Electr

subject and session name:  aq__3__2019-02-08-11-33-53_1stPart_1
1stPart
subject and session name:  aq__3__2019-02-08-12-11-34_2ndPart_1
2ndPart
As Nigel Baines,  secretary of the North Camp Matters Association,  points out,  it was a conscious decision to let North Camp run down at the expense of building up Farnborough to move closer to an overspill area.
As Niels Baines,  secretary of North Camp Association , points out ,  it was a conscious decision to let North Camp run down to for building upFornborough,  to come closer to an spilling area


An opportunity had arisen to include the site in the proposed sale of the adjacent land where the Territorial Army Voluntary Reserve Army (TAVRA) was located.
An opportunity had arisen to include the site in the proposed sale of the adjacent land , where the Territorial Army Voluntary Reserve ( TAVRA) was located .


She has a BA Hons and MA in Modern History from Royal Holloway,  University of London and an MA in Electronic Communications and

...

subject and session name:  ph__1__2019-01-28-13-49-14_2
She has a BA Hons and MA in Modern History from Royal Holloway,  University of London and an MA in Electronic Communications and Publishing from University College London.
She has a BA hons and MA in history from royal university,  university college Londonand MA in Electronic communication 


Neil Cugley was very pleased to get a home draw in the FA Cup 4th qualifying round against fellow Dr Martens League opponents Welling United,  the match is to be played on Saturday 27th October.
Neil Colgey was very pleased  to get a draw in home match fifth  game


The College combines a friendly and relaxed atmosphere with the tradition of vigorous argument which has made Cambridge English one of the most challenging in the University.
The college combines a friendly  and vigorous  atmosphere 


Representations of a Shrine on a Mycenaean Chariot Krater from Kalavasos-Ayios Dhimitrios,  Cyprus.
Representation  of a shrine 


An opportunity 

subject and session name:  ph__4__2019-02-18-16-19-28_2
Neck,  Topline,  Body Neck long,  lean and muscular with a slight arch to carry the head on high.
Neck ,  topline and body neckline 


You'll have to ask them.
You will have to ask them .


She's completely real that way.
She is completely real that way .


It is a database full of hairdressers and where you can find them.
It's a database full of hairdresser and a guide 


Shame it's nearly the summer holidays,  as I can't wait to put some of the tips into practice.
Shame its nearly the summer holidays


  subjectID block session  trial  CorrectLettersPerEffectiveTime
0        ph     4       2      0                        6.569454
1        ph     4       2      1                       10.861600
2        ph     4       2      2                       11.116800
3        ph     4       2      3                       11.790942
4        ph     4       2      4                       12.161086
subject and session name:  ph__5__2019-02-19

subject and session name:  rh__3_MS__2019-03-14-13-56-56_2
Landbrugets stigende forbrug af sprøjtemidler skal være slut.
Landbrugets stigende forbrug af sprøjtemidler skal være slut .


Kasserne fra pakkebåndet stakkes på pallen,  hvorefter den fyldte palle køres til udlevering.
Kasserne fra pakkebåndet stakkes på pallen,  hvorefter den fyldte Palle køres udyd  til levering.


Træningen foregik på eksercermarken ved kirkegården,  bag ved Odense Sygehus.
Træning foregik på eksercermarken ved kirkegården,  bag ved Odense Sygehus.


Så er webarkivering i Danmark ikke længere et nichefænomen,  som kun interesserer internetforskere.
Så er webarkivering i Danmark ikke længere et nichefænomen,  som kun interesserer internetforskere.


  subjectID block session  trial  CorrectLettersPerEffectiveTime
0        rh  3_MS       2      0                       10.622840
1        rh  3_MS       2      1                        6.085287
2        rh  3_MS       2      2                       13.644552
3 

She was about to get up and call it a day,  when she caught sight of a piece of paper lying on her desk.
She was about to get up and call it a day,  when she caugth sight of a piece of paper.


How long will the interview last?
How long will the interview last.


Set the alarm for 5am.
Set the alarm for 5am.


Sir,  your wit ambles well,  it goes easily.
Sir,  your wit ambles well,  it goesly easily.


Can easily be raised from seed.
Can easily be raised from seed.


  subjectID block session  trial  CorrectLettersPerEffectiveTime
0        ys     1       2      0                       24.455288
1        ys     1       2      1                       21.253464
2        ys     1       2      2                       22.280934
3        ys     1       2      3                       23.803181
4        ys     1       2      4                       24.977771
subject and session name:  ys__2__2019-01-16-15-18-50_1stPart_1
1stPart
subject and session name:  ys__2__2019-01-16-15-42-51_2ndPart_1
re

subject and session name:  ys__4_MS__2019-01-30-11-57-3_2
I was aware that the Eternal City has always been all things to all men.
I was aware that the Eternal City has always been all to all men 


He learned to rise early and work late at all times and in all weathers.
He learned to rise early and work late all days and in all weathers .


To,  make it to the top you have to be good with people.
To,  make it to the top you have to be good with people 


If in doubt about which level of course you think is appropriate,  please do not hesitate to ask.
If in doubt about which level of the course is appropriate,  please do not hesitate to ask 


This site uses frames - please update your browser.
Tip's site uses frames - please update your browser 


  subjectID block session  trial  CorrectLettersPerEffectiveTime
0        ys  4_MS       2      0                       12.750408
1        ys  4_MS       2      1                       12.602650
2        ys  4_MS       2      2              